In [ ]:
import numpy as np
import umap
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.io import loadmat
from tqdm import tqdm

In [ ]:
fs = 30000
fs_ms = 30
pre_ms = 1
post_ms = 10

In [ ]:
filename_list = ['waveform//NPXL_020_cplx_spks_channels_53_55_waveform.mat',
                 'waveform//NU_BRR_NPXL_003_session_1_WB_AP_translations_w_neural_ch42_waveform.mat',
                 'waveform//NU_BRR_NPXL_008_session_1_WB_AP_translations_w_neural_ch50_waveform.mat',
                 'waveform//NU_BRR_NPXL_009_session_1_WB_AP_translations_w_neural_ch56_waveform.mat',
                 'waveform//NU_BRR_NPXL_012_session_2_WB_AP_translations_w_neural_ch96_waveform.mat',
                 'waveform//NU_DRL_001_001_NU_DRL_001_001_Cplx_spk_waveform.mat',
                 'waveform//NU_DRL_002_cplx_spks_Cplx_spks_waveform.mat',
                 'waveform//NU_DRL_002_cplx_spks_WB_AP_translations_waveform.mat',
                 'waveform//NU_DRL_003_HoB_AP_translations_active_like_sorted_waveform.mat',
                 'waveform//NU_DRL_003_HoB_AP_translations_active_sorted_waveform.mat',
                 'waveform//NU_DRL_004_WB_AP_translations_active_like_sorted_waveform.mat',
                 'waveform//NU_DRL_004_WB_AP_translations_sine_waveform.mat',
                 'waveform//NU_DRL_018_WB_AP_translation2_waveform.mat'
                 ]

In [ ]:
cs_waveform = np.zeros((0,fs_ms*(pre_ms+post_ms)+1))
cs_idx = []

for i in range(len(filename_list)):
    data = loadmat(filename_list[i])
    w = np.array(data['w'])
    cs_waveform = np.concatenate([cs_waveform,w],axis=0)
    cs_idx.extend([i]*np.size(w,0))

# throw end sample point to match with simulated CS
cs_waveform = cs_waveform[:,:-1]

# cut 5ms
cs_waveform = cs_waveform[:,:180]


In [13]:
%matplotlib tk
plt.figure()
plt.subplot(211)
plt.plot(cs_waveform[np.array(cs_idx)==0,:].T)
plt.subplot(212)
plt.plot(np.diff(cs_waveform[np.array(cs_idx)==0,:],axis=-1).T)
plt.show()

In [6]:
sim_filename = 'sim_cs_waveform.mat'
sim_cs_waveform = np.zeros((0,fs_ms*(pre_ms+post_ms)+1))
sim_cs_idx = np.arange(10,210,10)

data = loadmat(sim_filename)
sim_cs_waveform = np.array(data['w'])

sim_cs_waveform = sim_cs_waveform[:,:180]

In [ ]:
# UMAP for recorded CS waveform
reducer_waveform = umap.UMAP()
embedding_waveform = reducer_waveform.fit_transform(cs_waveform)
sim_embedding_waveform = reducer_waveform.transform(sim_cs_waveform)

pca_waveform = PCA(2)
pc_waveform = pca_waveform.fit_transform(cs_waveform)
sim_pc_waveform = pca_waveform.transform(sim_cs_waveform)

In [ ]:
%matplotlib tk
# %matplotlib inline
plt.figure()
plt.title('UMAP waveform')
plt.scatter(
    embedding_waveform[:, 0],
    embedding_waveform[:, 1],c='r')
# plt.gca().set_aspect('equal', 'datalim')

for i, txt in enumerate(cs_idx):
    plt.annotate(txt, (embedding_waveform[i, 0], embedding_waveform[i, 1]))

plt.scatter(
    sim_embedding_waveform[:, 0],
    sim_embedding_waveform[:, 1],c='b')

for i, txt in enumerate(sim_cs_idx):
    plt.annotate(txt, (sim_embedding_waveform[i, 0], sim_embedding_waveform[i, 1]))

# PCA
plt.figure()
plt.title('PCA waveform')
plt.scatter(
    pc_waveform[:, 0],
    pc_waveform[:, 1],c='r')
# plt.gca().set_aspect('equal', 'datalim')

for i, txt in enumerate(cs_idx):
    plt.annotate(txt, (pc_waveform[i, 0], pc_waveform[i, 1]))

plt.scatter(
    sim_pc_waveform[:, 0],
    sim_pc_waveform[:, 1],c='b')

for i, txt in enumerate(sim_cs_idx):
    plt.annotate(txt, (sim_pc_waveform[i, 0], sim_pc_waveform[i, 1]))

In [ ]:
# 513*323
# cs_stft = np.zeros((0,513*323))

# cut 5ms
cs_stft = np.zeros((0,513*180))

for i in range(len(filename_list)):
    data = loadmat(filename_list[i])
    s = np.array(data['s'])
    s = s[:,:,:-1]

    # cut 5ms
    s = s[:,:,:180]

    cs_stft = np.concatenate([cs_stft,s.reshape(s.shape[0],-1)],axis=0)

In [ ]:
sim_filename = 'sim_cs_waveform.mat'
# sim_cs_stft = np.zeros((0,513*323))
sim_cs_stft = np.zeros((0,513*180))

data = loadmat(sim_filename)
s = np.array(data['s'])

# cut 5ms
s = s[:,:,:180]

sim_cs_stft = s.reshape(s.shape[0],-1)



In [ ]:
# UMAP for recorded CS stft
reducer_stft = umap.UMAP()
embedding_stft = reducer_stft.fit_transform(cs_stft)
sim_embedding_stft = reducer_stft.transform(sim_cs_stft)

pca_stft = PCA(2)
pc_stft = pca_stft.fit_transform(cs_stft)
sim_pc_stft = pca_stft.transform(sim_cs_stft)

In [ ]:
%matplotlib tk
# %matplotlib inline
plt.figure()
plt.title('UMAP STFT')
plt.scatter(
    embedding_stft[:, 0],
    embedding_stft[:, 1],c='r')
# plt.gca().set_aspect('equal', 'datalim')

for i, txt in enumerate(cs_idx):
    plt.annotate(txt, (embedding_stft[i, 0], embedding_stft[i, 1]))

plt.scatter(
    sim_embedding_stft[:, 0],
    sim_embedding_stft[:, 1],c='b')

for i, txt in enumerate(sim_cs_idx):
    plt.annotate(txt, (sim_embedding_stft[i, 0], sim_embedding_stft[i, 1]))

# PCA
plt.figure()
plt.title('PCA STFT')
plt.scatter(
    pc_stft[:, 0],
    pc_stft[:, 1],c='r')
# plt.gca().set_aspect('equal', 'datalim')

for i, txt in enumerate(cs_idx):
    plt.annotate(txt, (pc_stft[i, 0], pc_stft[i, 1]))

plt.scatter(
    sim_pc_stft[:, 0],
    sim_pc_stft[:, 1],c='b')

for i, txt in enumerate(sim_cs_idx):
    plt.annotate(txt, (sim_pc_stft[i, 0], sim_pc_stft[i, 1]))

In [ ]:
plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==0,:].T)
plt.title('0')
plt.show()

plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==1,:].T)
plt.title('1')
plt.show()

plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==2,:].T)
plt.title('2')
plt.show()

plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==4,:].T)
plt.title('4')
plt.show()

plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==5,:].T)
plt.title('5')
plt.show()

plt.figure()
plt.plot(cs_waveform[np.array(cs_idx)==7,:].T)
plt.title('7')
plt.show()

plt.figure()
plt.plot(sim_cs_waveform.T)
plt.show()